In [7]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize,word_tokenize
import numpy as np
import re
import numpy
from numpy import delete
import math
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
def read_article(para):
    file = open( r"C:\Users\user\001.txt",encoding='utf-8')
    filedata = file.read()
    print("Input file",filedata)
    print("\n\n")
    sentences= sent_tokenize(filedata)
#     print("Tokenized Sentences: ",sentences)
    total_num_sent=len(sentences)
#     print("total sentences",total_num_sent)
    sent=word_tokenize(filedata)
#     print(sentences)
    return sentences
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    #ps = PorterStemmer()
    
    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        #print(words)
        for word in words:
            word = word.lower()
            #word = ps.stem(word)
            #print(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
         
        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))
            
        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    sum=0
    text=[]
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)
            
        
#             print(tf_idf_table[word1])
        tf_idf_matrix[sent1]= tf_idf_table
#         print(tf_idf_matrix[sent1])
        
    return tf_idf_matrix
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    text = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        text[sent] = total_score_per_sentence / count_words_in_sentence
#     print("tf_idf score for all the sentences",text)   
    return text
def cosine_value(sentences):
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(sentences)


# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=[sentences])
    df
    cosine=cosine_similarity(df, df)
#     print(cosine)
    return cosine

def average_value(final_score): 

    return sum(final_score) / len(final_score)
    

def cal_pop_fitness(text, pop):
    n=[]
    num=[]
    arr=[]
    avg=[]
    arr1=[]
    final=[]
    final1=[]
    final2=[]
    final_score=[]
    file = open( r"C:\Users\user\001.txt",encoding='utf-8')
    filedata = file.read()
    sentences = sent_tokenize(filedata)
#     cal=len(sentence_score)/3
    num=5
    collection_set=[]
    solution_pop=18
    set_of=[]
    for i in range(solution_pop):
        for j in range(num):
            set_of.append(text[pop[i][j]])
      
        freq_matrix = _create_frequency_matrix(set_of)
        tf_matrix = _create_tf_matrix(freq_matrix)
    
        count_doc_per_words = _create_documents_per_words(freq_matrix)
        total_documents=len(set_of)
        idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
        tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    
        sentence_scores = _score_sentences(tf_idf_matrix)
        print("sent_score ---------",sentence_scores)
        average=average_value(sentence_scores.values())
        
        print("Average values---------",average)
        avg.append(average)
        print("\n")
        set_of=[]
    print(avg)
    return avg
    
   
            
    
    
    
    
    

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover
def generate_summary(para, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    

    # Step 1 - Read text anc split it
    sentences =  read_article(para)
    
    total_documents = len(sentences)
#     print(total_documents)
    freq_matrix = _create_frequency_matrix(sentences)
    #print("freqency_matrix",freq_matrix)

    
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print("tf_matrix:",tf_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
        
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print("idf_matrix",idf_matrix)

    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print("\n\n")
    #print("tf-idf: ",tf_idf_matrix)
    sentence_scores = _score_sentences(tf_idf_matrix)
    
    
#     print(sentence_scores)
#     text= list(sentence_scores.values())
#     print("Input_values--",text)
    text=list(sentence_scores.keys())
#     print("sentence of the documet: \t",text)
    numbers=[]
    for i in range(0, total_documents ):
        numbers.append(i)
    values_sent=list(numbers)
#     print(values_sent)
#     cal=len(sentence_score)/3
    num_weights=5
    sol_per_pop = len(sentence_scores)
    num_parents_mating = 6
   
    pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
   
    num_generations = 5
    for generation in range(num_generations):
        print("Generation : ", generation)
        new_population =numpy.random.choice(values_sent,pop_size)
        print("New population--",new_population)
        print("\n")
    # Measing the fitness of each chromosome in the population.
        fitness = cal_pop_fitness(text, new_population)
       
        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, num_parents_mating)
#         print("parents---",parents)
        print("\n")

    # Generating next generation using crossover.
        offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

#         print("offspring_crossover----", offspring_crossover)
        print("\n")
    # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover)
#         print("offspring_mutation---", offspring_mutation)
    # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
#         print("Best result : ", numpy.max(numpy.sum(new_population*text)))
#     print("Best result : ", numpy.max(fitness))#numpy.sum(new_population*text, axis=1)))

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
#     text=_tf_idf(tf_matrix, idf_matrix)
    fitness = cal_pop_fitness(text, new_population)
    
    #extracting the keys from values 
#     p= ([list(sentence_scores.keys())[list(sentence_scores.values()).index(fitness[i])]for i  in range(len(fitness))])
#     print(p)
#     f=open("output2.txt",'a')
#     f.write(str(p))

   

# Then return the index of that solution corresponding to the best fitness.
#     best_match_idx = numpy.where(fitness == numpy.max(fitness))
    print("\n")
    print ("Best solution : ", fitness)#values grater than average value
    print("\n")
    print("Best solution fitness : ", numpy.max(fitness))



generate_summary( "msft.txt", 2)

Input file Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to 